In [1]:



import os
# import urllib2
import pandas as pd
import urllib.request, json
from pprint import pprint
from pandas.io.json import json_normalize

In [2]:
with urllib.request.urlopen("https://covidtracking.com/api/states") as url:
    state_data = json.loads(url.read().decode())
#     pprint(state_data)

In [3]:
state_data_df = json_normalize(state_data[0:51]) 
state_data_df.head()

,state,positive,positiveScore,negativeScore,negativeRegularScore,commercialScore,grade,score,negative,pending,...,death,hospitalized,total,totalTestResults,posNeg,fips,dateModified,dateChecked,notes,hash
0,AK,337,1,1,1,1,A,4,11824,NaN,...,9,NaN,12161,12161,12161,02,2020-04-23T18:00:00Z,2020-04-23T19:36:00Z,"Please stop using the ""total"" field. Use ""tota...",51b52a9a3f56d10feb9e4a121f1d7f61d8496826
1,AL,5832,1,1,0,1,B,3,46863,NaN,...,197,768.0,52695,52695,52695,01,2020-04-23T04:00:00Z,2020-04-24T02:16:00Z,"Please stop using the ""total"" field. Use ""tota...",5a07c1d59e3a3083d37dbb6d05442509c9f48b94
2,AR,2465,1,1,1,1,A,4,29125,NaN,...,45,291.0,31590,31590,31590,05,2020-04-23T18:48:00Z,2020-04-23T19:43:00Z,"Please stop using the ""total"" field. Use ""tota...",9a82ff63b5fbdd28fdd37fd1cde5fc41b661ca7d
3,AZ,5769,1,1,0,1,B,3,52928,NaN,...,249,NaN,58697,58697,58697,04,2020-04-23T04:00:00Z,2020-04-23T19:47:00Z,"Please stop using the ""total"" field. Use ""tota...",30b6e0314763bb2bac3e283833d923f8d2093b52
4,CA,37369,1,1,0,1,B,3,444728,NaN,...,1469,NaN,482097,482097,482097,06,2020-04-22T04:00:00Z,2020-04-23T20:25:00Z,"Please stop using the ""total"" field. Use ""tota...",c66dcdd10e4a90f6254ae1adebae979f5f45f219


In [4]:
popul = os.path.join('./Resources/population.csv')
popul_df = pd.read_csv(popul)
popul_df.columns = ['state', 'population', 'pop (mil)']
# popul_df.dtypes

In [5]:
# concot_df = pd.concat([state_data_df, popul_df], axis=1, sort=False, join='inner')
concot_df2 = pd.merge(state_data_df,
                 popul_df[['state', 'pop (mil)']], on='state')

concot_df2.head()

,state,positive,positiveScore,negativeScore,negativeRegularScore,commercialScore,grade,score,negative,pending,...,hospitalized,total,totalTestResults,posNeg,fips,dateModified,dateChecked,notes,hash,pop (mil)
0,AK,337,1,1,1,1,A,4,11824,NaN,...,NaN,12161,12161,12161,02,2020-04-23T18:00:00Z,2020-04-23T19:36:00Z,"Please stop using the ""total"" field. Use ""tota...",51b52a9a3f56d10feb9e4a121f1d7f61d8496826,0.73
1,AL,5832,1,1,0,1,B,3,46863,NaN,...,768.0,52695,52695,52695,01,2020-04-23T04:00:00Z,2020-04-24T02:16:00Z,"Please stop using the ""total"" field. Use ""tota...",5a07c1d59e3a3083d37dbb6d05442509c9f48b94,4.90
2,AR,2465,1,1,1,1,A,4,29125,NaN,...,291.0,31590,31590,31590,05,2020-04-23T18:48:00Z,2020-04-23T19:43:00Z,"Please stop using the ""total"" field. Use ""tota...",9a82ff63b5fbdd28fdd37fd1cde5fc41b661ca7d,3.02
3,AZ,5769,1,1,0,1,B,3,52928,NaN,...,NaN,58697,58697,58697,04,2020-04-23T04:00:00Z,2020-04-23T19:47:00Z,"Please stop using the ""total"" field. Use ""tota...",30b6e0314763bb2bac3e283833d923f8d2093b52,7.28
4,CA,37369,1,1,0,1,B,3,444728,NaN,...,NaN,482097,482097,482097,06,2020-04-22T04:00:00Z,2020-04-23T20:25:00Z,"Please stop using the ""total"" field. Use ""tota...",c66dcdd10e4a90f6254ae1adebae979f5f45f219,39.51


In [6]:
state_covid_df = concot_df2[['state', 'pop (mil)', 'totalTestResults', 'negative','positive', 'death']]
state_covid_df.rename(columns = {'state':'State', 'pop (mil)':'Population (mil)', 'positive':'Positive Cases', 'negative':'Negative Tests',
                                 'pending':'Pending Tests', 'totalTestResults':'Total Tests','death':'Deaths'},inplace = True)
state_covid_df.set_index(['State'], inplace = True) 
state_covid_df.head()

C:\Users\jener\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Population (mil),Total Tests,Negative Tests,Positive Cases,Deaths
State,,,,,
AK,0.73,12161,11824,337,9
AL,4.90,52695,46863,5832,197
AR,3.02,31590,29125,2465,45
AZ,7.28,58697,52928,5769,249
CA,39.51,482097,444728,37369,1469


In [7]:
state_covid_df

,Population (mil),Total Tests,Negative Tests,Positive Cases,Deaths
State,,,,,
AK,0.73,12161,11824,337,9
AL,4.90,52695,46863,5832,197
AR,3.02,31590,29125,2465,45
AZ,7.28,58697,52928,5769,249
CA,39.51,482097,444728,37369,1469
CO,5.76,52324,41062,11262,552
CT,3.57,71497,48397,23100,1639
DC,0.71,15930,12569,3361,139
DE,0.97,16912,13604,3308,92


In [8]:
state_covid_df['% Positive Testing'] = (state_covid_df['Positive Cases'] / state_covid_df['Total Tests']) *100
state_covid_df['% Death Rate (Total Tests)'] = (state_covid_df['Deaths'] / state_covid_df['Total Tests']) *100
state_covid_df['% Death Rate (Positive Cases)'] = (state_covid_df['Deaths'] / state_covid_df['Positive Cases']) *100
state_covid_df['Tests per Million'] = (state_covid_df['Total Tests'] / state_covid_df['Population (mil)'])
state_covid_df['Positive per Million'] = (state_covid_df['Positive Cases'] / state_covid_df['Population (mil)'])
state_covid_df['Deaths per Million'] = (state_covid_df['Deaths'] / state_covid_df['Population (mil)'])


state_covid_df.round(2)

C:\Users\jener\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\jener\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jener\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,Population (mil),Total Tests,Negative Tests,Positive Cases,Deaths,% Positive Testing,% Death Rate (Total Tests),% Death Rate (Positive Cases),Tests per Million,Positive per Million,Deaths per Million
State,,,,,,,,,,,
AK,0.73,12161,11824,337,9,2.77,0.07,2.67,16658.90,461.64,12.33
AL,4.90,52695,46863,5832,197,11.07,0.37,3.38,10754.08,1190.20,40.20
AR,3.02,31590,29125,2465,45,7.80,0.14,1.83,10460.26,816.23,14.90
AZ,7.28,58697,52928,5769,249,9.83,0.42,4.32,8062.77,792.45,34.20
CA,39.51,482097,444728,37369,1469,7.75,0.30,3.93,12201.90,945.81,37.18
CO,5.76,52324,41062,11262,552,21.52,1.05,4.90,9084.03,1955.21,95.83
CT,3.57,71497,48397,23100,1639,32.31,2.29,7.10,20027.17,6470.59,459.10
DC,0.71,15930,12569,3361,139,21.10,0.87,4.14,22436.62,4733.80,195.77
DE,0.97,16912,13604,3308,92,19.56,0.54,2.78,17435.05,3410.31,94.85


In [9]:
state_covid_df['Tests per Million/Deaths'] = (state_covid_df['Tests per Million'] / state_covid_df['Deaths'])
state_covid_df['Tests per Million/Positive Cases'] = (state_covid_df['Tests per Million'] / state_covid_df['Positive Cases'])

state_covid_df.head()

C:\Users\jener\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\jener\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Population (mil),Total Tests,Negative Tests,Positive Cases,Deaths,% Positive Testing,% Death Rate (Total Tests),% Death Rate (Positive Cases),Tests per Million,Positive per Million,Deaths per Million,Tests per Million/Deaths,Tests per Million/Positive Cases
State,,,,,,,,,,,,,
AK,0.73,12161,11824,337,9,2.771154,0.074007,2.670623,16658.904110,461.643836,12.328767,1850.989346,49.432950
AL,4.90,52695,46863,5832,197,11.067464,0.373850,3.377915,10754.081633,1190.204082,40.204082,54.589247,1.843978
AR,3.02,31590,29125,2465,45,7.803102,0.142450,1.825558,10460.264901,816.225166,14.900662,232.450331,4.243515
AZ,7.28,58697,52928,5769,249,9.828441,0.424212,4.316173,8062.774725,792.445055,34.203297,32.380621,1.397604
CA,39.51,482097,444728,37369,1469,7.751345,0.304710,3.931066,12201.898254,945.811187,37.180461,8.306262,0.326525


In [11]:
state_covid_df.to_csv('./Resources/state_covid.csv')
